In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


# SELF JOINS

In [6]:
# sqlite 3 version

pd.read_sql_query(sql = '''
SELECT o1.* FROM orders o1
JOIN orders o2
ON o1.account_id = o2.account_id
AND o2.occurred_at > o1.occurred_at
AND o2.occurred_at <= date(o1.occurred_at, '+28 days')
ORDER BY o1.account_id, o1.occurred_at
;
''', con=conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,4307,1001,2015-11-05 03:25:21,506,612,203,1321,2524.94,4583.88,1648.36,8757.18
1,4308,1001,2015-12-04 04:01:09,526,597,287,1410,2624.74,4471.53,2330.44,9426.71
2,4309,1001,2016-01-02 00:59:09,566,645,194,1405,2824.34,4831.05,1575.28,9230.67
3,4310,1001,2016-02-01 19:07:32,473,595,212,1280,2360.27,4456.55,1721.44,8538.26
4,6,1001,2016-03-02 15:29:32,103,24,46,173,513.97,179.76,373.52,1067.25
...,...,...,...,...,...,...,...,...,...,...,...
2545,4300,4501,2016-06-29 03:57:11,104,14,31,149,518.96,104.86,251.72,875.54
2546,6909,4501,2016-07-29 19:58:32,5,91,96,192,24.95,681.59,779.52,1486.06
2547,4302,4501,2016-08-27 00:48:17,180,11,24,215,898.20,82.39,194.88,1175.47
2548,6911,4501,2016-11-22 06:52:22,63,67,81,211,314.37,501.83,657.72,1473.92


In [7]:
pd.read_sql_query(sql = '''
SELECT o1.* FROM orders o1
JOIN orders o2
ON o1.account_id = o2.account_id
AND o2.occurred_at > o1.occurred_at
AND o2.occurred_at <= o1.occurred_at + INTERVAL '28 days'
;
''', con=conn)

DatabaseError: Execution failed on sql '
SELECT o1.* FROM orders o1
JOIN orders o2
ON o1.account_id = o2.account_id
AND o2.occurred_at > o1.occurred_at
AND o2.occurred_at <= o1.occurred_at + INTERVAL '28 days'
;
': near "'28 days'": syntax error

Expert Tip


This comes up pretty commonly in job interviews. Self JOIN logic can be pretty tricky -- you can see here that our join has three conditional statements. It is important to pause and think through each step when joining a table to itself.

In [8]:
# sqlite 3 version

pd.read_sql_query(sql = '''
WITH t1 AS(
SELECT o1.account_id AS o1_account_id, o1.occurred_at AS o1_occurred_at,
o2.account_id AS o2_account_id, o2.occurred_at AS o2_occurred_at
FROM orders o1
LEFT JOIN orders o2
ON o1.account_id = o2.account_id
AND o2.occurred_at > o1.occurred_at
AND o2.occurred_at <= date(o1.occurred_at, '+28 days')
ORDER BY o1.account_id, o1.occurred_at)

SELECT *,
LAG(o1_occurred_at) OVER (PARTITION BY o1_account_id ORDER BY o1_occurred_at) AS lag,
JulianDay(o1_occurred_at) - JulianDay(LAG(o1_occurred_at) OVER (PARTITION BY o1_account_id ORDER BY o1_occurred_at)) AS dif FROM t1;
''', con=conn)

,o1_account_id,o1_occurred_at,o2_account_id,o2_occurred_at,lag,dif
0,1001,2015-10-06 17:31:14,NaN,None,None,NaN
1,1001,2015-11-05 03:25:21,1001.0,2015-11-05 03:34:33,2015-10-06 17:31:14,29.412581
2,1001,2015-11-05 03:34:33,NaN,None,2015-11-05 03:25:21,0.006389
3,1001,2015-12-04 04:01:09,1001.0,2015-12-04 04:21:55,2015-11-05 03:34:33,29.018472
4,1001,2015-12-04 04:21:55,NaN,None,2015-12-04 04:01:09,0.014421
...,...,...,...,...,...,...
6907,4501,2016-10-24 08:50:37,NaN,None,2016-09-25 01:44:03,29.296227
6908,4501,2016-11-22 06:52:22,4501.0,2016-11-22 06:57:04,2016-10-24 08:50:37,28.917882
6909,4501,2016-11-22 06:57:04,NaN,None,2016-11-22 06:52:22,0.003264
6910,4501,2016-12-21 13:30:42,4501.0,2016-12-21 13:43:26,2016-11-22 06:57:04,29.273356


In [10]:
pd.read_sql_query(sql = '''
SELECT o1.account_id AS o1_account_id, o1.occurred_at AS o1_occurred_at,
o2.account_id AS o2_account_id, o2.occurred_at AS o2_occurred_at
FROM orders o1
JOIN orders o2
ON o1_account_id = o2_account_id
AND o2_occurred_at > o1_occurred_at
AND o2_occurred_at <= date(o1_occurred_at, '+28 days')
ORDER BY o1_account_id, o1_occurred_at
''', con=conn)

,o1_account_id,o1_occurred_at,o2_account_id,o2_occurred_at
0,1001,2015-11-05 03:25:21,1001,2015-11-05 03:34:33
1,1001,2015-12-04 04:01:09,1001,2015-12-04 04:21:55
2,1001,2016-01-02 00:59:09,1001,2016-01-02 01:18:24
3,1001,2016-02-01 19:07:32,1001,2016-02-01 19:27:27
4,1001,2016-03-02 15:29:32,1001,2016-03-02 15:40:29
...,...,...,...,...
2545,4501,2016-06-29 03:57:11,4501,2016-06-29 04:03:39
2546,4501,2016-07-29 19:58:32,4501,2016-07-29 20:06:39
2547,4501,2016-08-27 00:48:17,4501,2016-08-27 00:58:11
2548,4501,2016-11-22 06:52:22,4501,2016-11-22 06:57:04


In [11]:
pd.read_sql_query(sql = '''
WITH t1 AS(
SELECT o1.account_id AS o1_account_id, o1.occurred_at AS o1_occurred_at,
o2.account_id AS o2_account_id, o2.occurred_at AS o2_occurred_at
FROM orders o1
LEFT JOIN orders o2
ON o1_account_id = o2_account_id
AND o2_occurred_at > o1_occurred_at
AND o2_occurred_at <= date(o1_occurred_at, '+28 days')
ORDER BY o1_account_id, o1_occurred_at)

SELECT COUNT(o2_account_id) FROM t1;
''', con=conn)

,COUNT(o2_account_id)
0,2550


In [12]:
# sqlite 3 version

pd.read_sql_query(sql = '''
WITH t1 AS(
SELECT o1.account_id AS o1_account_id, o1.occurred_at AS o1_occurred_at,
o2.account_id AS o2_account_id, o2.occurred_at AS o2_occurred_at
FROM orders o1
LEFT JOIN orders o2
ON o1_account_id = o2_account_id
AND o2_occurred_at > o1_occurred_at
AND o2_occurred_at <= date(o1_occurred_at, '+28 days')
ORDER BY o1_account_id, o1_occurred_at)

SELECT *,
LAG(o1_occurred_at) OVER (PARTITION BY o1_account_id ORDER BY o1_occurred_at) AS lag,
JulianDay(o1_occurred_at) - JulianDay(LAG(o1_occurred_at) OVER (PARTITION BY o1_account_id ORDER BY o1_occurred_at)) AS dif
FROM t1
ORDER BY JulianDay(o1_occurred_at) - JulianDay(LAG(o1_occurred_at) OVER (PARTITION BY o1_account_id ORDER BY o1_occurred_at)) DESC;
''', con=conn)

,o1_account_id,o1_occurred_at,o2_account_id,o2_occurred_at,lag,dif
0,1701,2014-03-12 22:32:17,1701.0,2014-03-12 22:34:36,2014-01-11 11:13:37,60.471296
1,2281,2016-09-19 18:08:35,2281.0,2016-09-19 18:31:46,2016-07-21 09:22:31,60.365324
2,3901,2017-01-01 11:30:00,3901.0,2017-01-01 11:31:53,2016-11-02 05:50:38,60.235671
3,2921,2016-06-20 16:46:06,2921.0,2016-06-20 17:12:17,2016-04-21 16:17:46,60.019676
4,2351,2016-12-15 00:16:07,NaN,None,2016-10-16 08:28:39,59.657963
...,...,...,...,...,...,...
6907,4461,2014-08-25 04:07:47,4461.0,2014-08-25 04:09:22,None,NaN
6908,4471,2016-12-05 04:59:53,4471.0,2016-12-05 05:10:54,None,NaN
6909,4481,2016-10-08 07:36:28,4481.0,2016-10-08 07:38:46,None,NaN
6910,4491,2013-12-08 06:34:23,NaN,None,None,NaN


In [13]:
# sqlite 3 version

pd.read_sql_query(sql = '''
WITH t1 AS(SELECT o1.id AS o1_id,
       o1.account_id AS o1_account_id,
       o1.occurred_at AS o1_occurred_at,
       o2.id AS o2_id,
       o2.account_id AS o2_account_id,
       o2.occurred_at AS o2_occurred_at
  FROM orders o1
 LEFT JOIN orders o2
   ON o1.account_id = o2.account_id
  AND o2.occurred_at > o1.occurred_at
  AND o2.occurred_at <= date(o1.occurred_at, '+28 days')
ORDER BY o1.account_id, o1.occurred_at)

SELECT *,
LAG(o1_occurred_at) OVER (PARTITION BY o1_account_id ORDER BY o1_occurred_at) AS lag,
JulianDay(o1_occurred_at) - JulianDay(LAG(o1_occurred_at) OVER (PARTITION BY o1_account_id ORDER BY o1_occurred_at)) AS dif
FROM t1
ORDER BY JulianDay(o1_occurred_at) - JulianDay(LAG(o1_occurred_at) OVER (PARTITION BY o1_account_id ORDER BY o1_occurred_at)) DESC;
''', con=conn)

,o1_id,o1_account_id,o1_occurred_at,o2_id,o2_account_id,o2_occurred_at,lag,dif
0,1010,1701,2014-03-12 22:32:17,4931.0,1701.0,2014-03-12 22:34:36,2014-01-11 11:13:37,60.471296
1,5379,2281,2016-09-19 18:08:35,1706.0,2281.0,2016-09-19 18:31:46,2016-07-21 09:22:31,60.365324
2,3592,3901,2017-01-01 11:30:00,6472.0,3901.0,2017-01-01 11:31:53,2016-11-02 05:50:38,60.235671
3,5871,2921,2016-06-20 16:46:06,2559.0,2921.0,2016-06-20 17:12:17,2016-04-21 16:17:46,60.019676
4,1803,2351,2016-12-15 00:16:07,NaN,NaN,None,2016-10-16 08:28:39,59.657963
...,...,...,...,...,...,...,...,...
6907,6868,4461,2014-08-25 04:07:47,4248.0,4461.0,2014-08-25 04:09:22,None,NaN
6908,4278,4471,2016-12-05 04:59:53,6893.0,4471.0,2016-12-05 05:10:54,None,NaN
6909,4279,4481,2016-10-08 07:36:28,6894.0,4481.0,2016-10-08 07:38:46,None,NaN
6910,6896,4491,2013-12-08 06:34:23,NaN,NaN,None,None,NaN
